# VacationPy
---

## Starter Code to Import Libraries and Load the Weather and Coordinates Data

In [44]:
!pip install gmaps

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.1/1.1 MB 3.8 MB/s eta 0:00:0000:0100:01
  Preparing metadata (setup.py) ... done
  Created wheel for gmaps: filename=gmaps-0.9.0-py2.py3-none-any.whl size=2076090 sha256=5a63fd7814f404db031147d91405c0d9d503ef3eb2bf60f3eb3fe247eebfd618
  Stored in directory: /Users/nicoleharris/Library/Caches/pip/wheels/66/ab/29/3032938612273665f0fa4d2fab4e3ecfe86480eaaa40aaee91
Successfully built gmaps


In [45]:
# Dependencies and Setup
import hvplot.pandas
import pandas as pd
import requests
import matplotlib.pyplot as plt
import numpy as np
import requests
import gmaps
import os
import json

# Import API key
from api_keys import geoapify_key

In [42]:
# Load the CSV file created in Part 1 into a Pandas DataFrame
city_data_df = pd.read_csv("output_data/cities.csv")

# Display sample data
city_data_df.head()

,City_ID,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date
0,0,longyearbyen,78.2186,15.6401,268.06,68,40,5.14,SJ,1684369231
1,1,yellowknife,62.4560,-114.3525,287.44,41,20,6.17,CA,1684369231
2,2,blackmans bay,-43.0167,147.3167,281.89,78,53,3.48,AU,1684369231
3,3,ribeira grande,38.5167,-28.7000,291.36,88,100,2.65,PT,1684369231
4,4,vilyuchinsk,52.9306,158.4028,279.30,84,100,3.83,RU,1684369231


---

### Step 1: Create a map that displays a point for every city in the `city_data_df` DataFrame. The size of the point should be the humidity in each city.

In [46]:
# Import the required libraries
import gmaps
import gmaps.datasets
# Create a map that displays a point for every city in the city_data_df DataFrame
locations = city_data_df[["Lat", "Lng"]]
humidity = city_data_df["Humidity"]
fig = gmaps.figure(center=(30.0, 31.0), zoom_level=1.5)

# Create a heatmap layer with the humidity data
heat_layer = gmaps.heatmap_layer(locations, weights=humidity, dissipating=False, max_intensity=300, point_radius=4)

# Add the heatmap layer to the map
fig.add_layer(heat_layer)

# Display the map
fig

Figure(layout=FigureLayout(height='420px'))

In [49]:


# Iterate through the DataFrame.
for index, row in hotel_df.iterrows():
    
    # Get the latitude and longitude.
    lat = row["Lat"]
    lng = row["Lng"]

    # Add the latitude and longitude to location key for the params dictionary.
    params["location"] = f"{lat},{lng}"

    # Use the search term: "lodging" and our latitude and longitude.
    base_url = "https://maps.googleapis.com/maps/api/place/nearbysearch/json"
    
    # Make request and get the JSON data from the search.
    hotels = requests.get(base_url, params=params).json()
    
    # Grab the first hotel from the results and store the name.
    try:
        hotel_df.loc[index, "Hotel Name"] = hotels["results"][0]["name"]
    except (IndexError):
        print("Hotel not found... skipping.")

hotel_df.head(10)

,City_ID,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date,Hotel Name


### Step 2: Narrow down the `city_data_df` DataFrame to find your ideal weather condition

In [50]:
# Define your ideal weather conditions
ideal_conditions = {
    "Max Temp": 70,  # Maximum temperature in Fahrenheit
    "Humidity": 50,  # Humidity percentage
    "Cloudiness": 20,  # Cloudiness percentage
    "Wind Speed": 10  # Wind speed in mph
}

# Filter the DataFrame based on the ideal conditions
filtered_df = city_data_df[
    (city_data_df["Max Temp"] <= ideal_conditions["Max Temp"]) &
    (city_data_df["Humidity"] <= ideal_conditions["Humidity"]) &
    (city_data_df["Cloudiness"] <= ideal_conditions["Cloudiness"]) &
    (city_data_df["Wind Speed"] <= ideal_conditions["Wind Speed"])
].copy()

# Iterate through the filtered cities and retrieve their coordinates
city_coordinates = []
for city in filtered_df["City"]:
    coordinates = geocode_city(city)
    if coordinates:
        city_coordinates.append(coordinates)

# Display the city coordinates
for city, coordinates in zip(filtered_df["City"], city_coordinates):
    print(f"City: {city}, Coordinates: {coordinates}")



### Step 3: Create a new DataFrame called `hotel_df`.

In [51]:
import pandas as pd

# Create a copy of the filtered DataFrame
hotel_df = filtered_df.copy()

# Add an empty column "Hotel Name"
hotel_df["Hotel Name"] = ""

# Display sample data
hotel_df.head()


,City_ID,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date,Hotel Name


### Step 4: For each city, use the Geoapify API to find the first hotel located within 10,000 metres of your coordinates.

In [52]:
# Set parameters to search for a hotel
radius = 10000
params = { "type": "hotel",
    "radius": radius,
    "limit": 1
    
}

# Print a message to follow up the hotel search
print("Starting hotel search")

# Iterate through the hotel_df DataFrame
for index, row in hotel_df.iterrows():
    # get latitude, longitude from the DataFrame
    # lat = row['Lat']
    lng = row['Lng']
    
    
    # Add filter and bias parameters with the current city's latitude and longitude to the params dictionary
    params["filter"] = lat
    params["bias"] = lng
    
    # Set base URL
    base_url = "https://api.geoapify.com/v2/places"


    # Make and API request using the params dictionaty
    name_address = requests.get(base_url, params=params)
    
    # Convert the API response to JSON format
    name_address = name_address = response.json()

    
    # Grab the first hotel from the results and store the name in the hotel_df DataFrame
    try:
        hotel_df.loc[index, "Hotel Name"] = name_address["features"][0]["properties"]["name"]
    except (KeyError, IndexError):
        # If no hotel is found, set the hotel name as "No hotel found".
        hotel_df.loc[index, "Hotel Name"] = "No hotel found"
        
    # Log the search results
    print(f"{hotel_df.loc[index, 'City']} - nearest hotel: {hotel_df.loc[index, 'Hotel Name']}")

# Display sample data
hotel_df

Starting hotel search


,City_ID,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date,Hotel Name


### Step 5: Add the hotel name and the country as additional information in the hover message for each city in the map.

In [53]:
print(hotel_df)


Empty DataFrame
Columns: [City_ID, City, Lat, Lng, Max Temp, Humidity, Cloudiness, Wind Speed, Country, Date, Hotel Name]
Index: []


In [54]:
print(mean_lat, mean_lng)


nan nan
